In [4]:
class MultiScaleMultiStageCNN:
    """Multi-scale, Multi-stage Deep Learning Model for Copy-Move Forgery Detection"""
    def __init__(self, input_shape=(256, 256, 3)):
        self.input_shape = input_shape
        self.model = None
        self.feature_extractor = None
        self.build_model()

    def conv_block(self, inputs, filters, kernel_size=3, activation='relu'):
        x = Conv2D(filters, kernel_size, padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Activation(activation)(x)
        x = Conv2D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x

    def encoder_block(self, inputs, filters):
        conv = self.conv_block(inputs, filters)
        pool = MaxPooling2D(pool_size=(2, 2))(conv)
        return conv, pool

    def decoder_block(self, inputs, skip_features, filters):
        upsample = Conv2DTranspose(filters, (2, 2), strides=2, padding='same')(inputs)
        concat = concatenate([upsample, skip_features])
        conv = self.conv_block(concat, filters)
        return conv

    def build_model(self):
        inputs = Input(shape=self.input_shape)

        # Custom layers for resizing
        class ResizeLayer(tf.keras.layers.Layer):
            def __init__(self, target_size, **kwargs):
                super(ResizeLayer, self).__init__(**kwargs)
                self.target_size = target_size

            def call(self, inputs):
                return tf.image.resize(inputs, self.target_size)

        scale1 = inputs
        scale2 = ResizeLayer([self.input_shape[0]//2, self.input_shape[1]//2])(inputs)
        scale2 = ResizeLayer([self.input_shape[0], self.input_shape[1]])(scale2)
        scale3 = ResizeLayer([self.input_shape[0]//4, self.input_shape[1]//4])(inputs)
        scale3 = ResizeLayer([self.input_shape[0], self.input_shape[1]])(scale3)

        multi_scale_input = concatenate([scale1, scale2, scale3])

        conv1, pool1 = self.encoder_block(multi_scale_input, 64)
        conv2, pool2 = self.encoder_block(pool1, 128)
        conv3, pool3 = self.encoder_block(pool2, 256)
        conv4, pool4 = self.encoder_block(pool3, 512)

        bridge = self.conv_block(pool4, 1024)
        bridge = Dropout(0.5)(bridge)

        dec4 = self.decoder_block(bridge, conv4, 512)
        dec3 = self.decoder_block(dec4, conv3, 256)
        dec2 = self.decoder_block(dec3, conv2, 128)
        dec1 = self.decoder_block(dec2, conv1, 64)

        outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(dec1)
        self.model = Model(inputs=inputs, outputs=outputs)
        self.model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )
        self.feature_extractor = Model(inputs=inputs, outputs=[conv1, conv2, conv3, conv4])

    def extract_features(self, image):
        if len(image.shape) == 3:
            image = np.expand_dims(image, axis=0)
        features = self.feature_extractor.predict(image, verbose=0)
        return features

    def predict_forgery_mask(self, image):
        if len(image.shape) == 3:
            image = np.expand_dims(image, axis=0)
        mask = self.model.predict(image, verbose=0)
        return mask[0, :, :, 0]

In [2]:
class CopyMoveForgeryDetector:
    """Complete Copy-Move Forgery Detection System"""
    def __init__(self):
        self.cnn_model = MultiScaleMultiStageCNN()
        self.block_size = 16
        self.overlap = 8
        self.similarity_threshold = 0.85

    def preprocess_image(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not load image")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image, (256, 256))
        image_normalized = image_resized.astype(np.float32) / 255.0
        return image, image_resized, image_normalized

    def block_based_detection(self, image):
        h, w = image.shape[:2]
        blocks = []
        positions = []
        for i in range(0, h - self.block_size + 1, self.overlap):
            for j in range(0, w - self.block_size + 1, self.overlap):
                block = image[i:i+self.block_size, j:j+self.block_size]
                if block.shape[0] == self.block_size and block.shape[1] == self.block_size:
                    gray_block = cv2.cvtColor(block, cv2.COLOR_RGB2GRAY) if len(block.shape) == 3 else block
                    blocks.append(gray_block.flatten())
                    positions.append((i, j))
        if len(blocks) < 2:
            return np.zeros((h, w), dtype=np.uint8)
        blocks = np.array(blocks)
        distances = cdist(blocks, blocks, metric='euclidean')
        forgery_mask = np.zeros((h, w), dtype=np.uint8)
        for i in range(len(blocks)):
            for j in range(i + 1, len(blocks)):
                pos1, pos2 = positions[i], positions[j]
                distance_between_blocks = np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)
                if (distances[i, j] < (1 - self.similarity_threshold) * np.max(distances) and
                    distance_between_blocks > self.block_size * 2):
                    forgery_mask[pos1[0]:pos1[0]+self.block_size, pos1[1]:pos1[1]+self.block_size] = 255
                    forgery_mask[pos2[0]:pos2[0]+self.block_size, pos2[1]:pos2[1]+self.block_size] = 255
        return forgery_mask

    def deep_learning_detection(self, image_normalized):
        forgery_mask = self.cnn_model.predict_forgery_mask(image_normalized)
        binary_mask = (forgery_mask > 0.5).astype(np.uint8) * 255
        return binary_mask

    def hybrid_detection(self, image, image_normalized):
        traditional_mask = self.block_based_detection(image)
        dl_mask = self.deep_learning_detection(image_normalized)
        if dl_mask.shape != traditional_mask.shape:
            dl_mask = cv2.resize(dl_mask, (traditional_mask.shape[1], traditional_mask.shape[0]))
        combined_mask = cv2.bitwise_and(traditional_mask, dl_mask)
        kernel = np.ones((5, 5), np.uint8)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
        return traditional_mask, dl_mask, combined_mask

    def detect_forgery(self, image_path):
        try:
            original_image, resized_image, normalized_image = self.preprocess_image(image_path)
            traditional_mask, dl_mask, combined_mask = self.hybrid_detection(resized_image, normalized_image)
            forgery_pixels = np.sum(combined_mask > 0)
            total_pixels = combined_mask.shape[0] * combined_mask.shape[1]
            confidence_score = min(forgery_pixels / total_pixels * 100, 100)
            is_forged = confidence_score > 5.0
            results = {
                'is_forged': is_forged,
                'confidence_score': confidence_score,
                'original_image': original_image,
                'processed_image': resized_image,
                'traditional_mask': traditional_mask,
                'dl_mask': dl_mask,
                'combined_mask': combined_mask
            }
            return results
        except Exception as e:
            raise Exception(f"Error in forgery detection: {str(e)}")


In [6]:
!pip install keras


  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached h5py-3.14.0-cp311-cp311-win_amd64.whl.metadata (2.7 kB)
  Using cached optree-0.16.0-cp311-cp311-win_amd64.whl.metadata (31 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 645.7 kB/s eta 0:00:02
   ---------------------- ----------------- 0.8/1.4 MB 819.2 kB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 819.2 kB/s eta 0:00:01
   ------------

In [8]:
!pip install tensorflow

  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/375.9 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-health-checking 1.73.0 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.
grpcio-tools 1.73.0 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.


In [9]:
import pickle
import os
model_path=model_path = os.path.join("artifacts", "model.pkl")
with open(model_path, 'rb') as f:
        model = pickle.load(f)

ImportError: Traceback (most recent call last):
  File "C:\Users\itsar\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.